### Task Schedule:
1. 訓練模型,調整參數(預計使用lgb，速度較快)(更:使用catboost,效果較好)
2. 嘗試使用不同模型,做Ensamble(blending, stacking)
3. Anomaly detection

### 注意事項:
1. 因為test data和train data時間不相關,在驗證時採取前60天訓練61~90天驗證,但仍需小心時間差異造成的影響
2. Anomaly detection: 看這類的模型能不能取代boosting(似乎是不行，盜刷數據並沒有那麼Anomaly）,但可以嘗試將Anomaly結果當成新feature

### <font color=green>Results:</font>

#### Catboost:
    * FE1~4,catboost訓練 validation:0.5, LB:0.55
    * FE1,3,4 validation:0.5313149, LB:0.6(th=0.37), default parameter

#### LGB:
    * 不做處理,直接丟lgb訓練 leaderboard score:0.45

## 讀取,轉換字串成可以訓練的資料

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import math

import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier, Pool

%matplotlib inline
data_path = '../data'

random_seed = 20
import json
path = '../code/para_dict/data_list.json'
with open(path,'r',encoding='utf-8') as f:
    para = json.loads(f.read())
    
data_list= para['data_list_FE_AN']
delete_list = para['delete_list_overfit1']

## 除掉一些可能會overfit,distribution不同,受時間影響大的feature

delete_list1 = ['bacno','locdt','loctm','cano','fraud_ind']
delete_list2 = ['mchno','acqic','mcc']
delete_list3 = ['stocn','scity','csmcu']
delete_list4 = ['iterm']
delete_list6 = ['mchno_fraud_mean','mcc_fraud_mean','acqic_fraud_mean']
delete_list7 = ['bacno_locdt_skew','bacno_locdt_kurt','cano_locdt_skew','cano_locdt_kurt']
delete_list8 = ['bacno_lastlocdt','cano_lastlocdt']

delete_list5 = ['contp','etymd','hcefg','insfg','ovrlt','flbmk','flg_3dsmk']
# bacno_cano_nunique
# txkey

In [2]:
def load_data(data_list):
    data=[]
    for d in data_list:
        x = pd.read_csv('../data/preprocess/{}'.format(d))
        x_null = x.isnull().sum()
        
        print('\n',d,x.shape)
        print("Null columns:\n",x_null[x_null>0])

        if (d=='FE_data1.csv') or (d=='FE_data2.csv'):
            x.fillna(value=-1,inplace=True)

        data.append(x)

    all_data = pd.concat(data,axis=1)
    all_data_numsum = all_data.isnull().sum()
    print('ALL data shape:',all_data.shape)
    print('ALL data null:')
    print(all_data_numsum[all_data_numsum>0])
    return all_data

all_data = load_data(data_list)


 raw_data.csv (1943452, 23)
Null columns:
 fraud_ind    421665
dtype: int64

 FE_data1.csv (1943452, 56)
Null columns:
 cano_conam_skew      92612
cano_conam_kurt     155720
cano_conam_var       38678
bacno_locdt_skew     58303
bacno_locdt_kurt    101191
cano_locdt_skew      92612
cano_locdt_kurt     155720
dtype: int64

 FE_data2.csv (1943452, 30)
Null columns:
 Series([], dtype: int64)

 FE_data3.csv (1943452, 9)
Null columns:
 Series([], dtype: int64)

 pca_feature.csv (1943452, 5)
Null columns:
 Series([], dtype: int64)

 isolationtree_feature.csv (1943452, 1)
Null columns:
 Series([], dtype: int64)

 kmeans_feature.csv (1943452, 3)
Null columns:
 Series([], dtype: int64)
ALL data shape: (1943452, 127)
ALL data null:
fraud_ind    421665
dtype: int64


In [3]:
category_list=['csmcu','hcefg','stscd','scity','stocn','mcc','acqic','mchno','etymd','contp']
all_data[category_list]=all_data[category_list].astype('category')

In [4]:
## 切三種不同的訓練集驗證

X_train1 = all_data[all_data['locdt']<=60].drop(columns=delete_list)
y_train1 = all_data[all_data['locdt']<=60]['fraud_ind']
X_test1 = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)].drop(columns=delete_list)
y_test1 = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)]['fraud_ind']

X_train2 = all_data[all_data['locdt']<=45].drop(columns=delete_list)
y_train2 = all_data[all_data['locdt']<=45]['fraud_ind']
X_test2 = all_data[(all_data['locdt']>45) & (all_data['locdt']<=90)].drop(columns=delete_list)
y_test2 = all_data[(all_data['locdt']>45) & (all_data['locdt']<=90)]['fraud_ind']

X_train3 = all_data[all_data['locdt']<=30].drop(columns=delete_list)
y_train3 = all_data[all_data['locdt']<=30]['fraud_ind']
X_test3 = all_data[(all_data['locdt']>30) & (all_data['locdt']<=90)].drop(columns=delete_list)
y_test3 = all_data[(all_data['locdt']>30) & (all_data['locdt']<=90)]['fraud_ind']


test_data_txkey = all_data[all_data['locdt']>90]['txkey'].copy().values
X_train_all = all_data[all_data['locdt']<=90].drop(columns=delete_list) 
y_train_all = all_data[all_data['locdt']<=90]['fraud_ind'] 

X_test_all = all_data[all_data['locdt']>90].drop(columns=delete_list) 
# y_test_all = all_data[all_data['locdt']>90]['fraud_ind'] 

## Train on catboost
* https://catboost.ai/docs/concepts/python-reference_parameters-list.html
* 研究有哪些可以用的function

In [5]:
categorical_features_indices = np.where(X_train1.columns.isin(category_list))[0]
print(X_train1.dtypes[categorical_features_indices])


acqic    category
contp    category
csmcu    category
etymd    category
hcefg    category
mcc      category
mchno    category
scity    category
stocn    category
stscd    category
dtype: object


In [6]:
param_cat={
    'loss_function':'Logloss',
    'eval_metric':'F1',
    
    'iterations':2000,
    'learning_rate':0.1,
    'l2_leaf_reg':3,
    'bagging_temperature':1,
#     'sampling_frequency':'PerTreeLevel',
    
    'depth':6,
    'one_hot_max_size':300,
    
#     'min_data_in_leaf':1,
#     'max_leaves':31,
#     'task_type':"GPU",
#     'devices':1',
#     'cat_features':categorical_features_indices,
    'rsm':1,
    'scale_pos_weight':1,
    'target_border':0.5,
    'random_seed':random_seed,
    'verbose':True    
}

In [7]:
model = CatBoostClassifier(**param_cat)

model.fit(
    X_train1, y_train1,
    cat_features=categorical_features_indices,    
    eval_set=(X_test1, y_test1),
    early_stopping_rounds=200,
#     use_best_model=True,
    silent=False,
#     plot=True,
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())


# preds_class = model.predict(test_data)
# preds_proba = model.predict_proba(test_data)

0:	learn: 0.5886076	test: 0.3337593	best: 0.3337593 (0)	total: 1.31s	remaining: 1h 5m 19s
1:	learn: 0.5576330	test: 0.3169252	best: 0.3337593 (0)	total: 2.45s	remaining: 1h 1m 14s
2:	learn: 0.5664179	test: 0.3198565	best: 0.3337593 (0)	total: 3.65s	remaining: 1h 44s
3:	learn: 0.5810301	test: 0.3394656	best: 0.3394656 (3)	total: 4.78s	remaining: 59m 43s
4:	learn: 0.5978199	test: 0.3518200	best: 0.3518200 (4)	total: 5.98s	remaining: 59m 43s
5:	learn: 0.6108130	test: 0.3797468	best: 0.3797468 (5)	total: 7.06s	remaining: 58m 42s
6:	learn: 0.6143139	test: 0.3883820	best: 0.3883820 (6)	total: 8.04s	remaining: 57m 17s
7:	learn: 0.6163790	test: 0.3970066	best: 0.3970066 (7)	total: 9.19s	remaining: 57m 16s
8:	learn: 0.6230342	test: 0.4023032	best: 0.4023032 (8)	total: 10.3s	remaining: 57m 3s
9:	learn: 0.6251876	test: 0.4051511	best: 0.4051511 (9)	total: 11.4s	remaining: 57m 2s
10:	learn: 0.6249850	test: 0.4076628	best: 0.4076628 (10)	total: 12.5s	remaining: 56m 38s
11:	learn: 0.6273593	test: 0.

92:	learn: 0.7516228	test: 0.5279587	best: 0.5283204 (82)	total: 1m 40s	remaining: 52m 19s
93:	learn: 0.7521074	test: 0.5285521	best: 0.5285521 (93)	total: 1m 41s	remaining: 52m 24s
94:	learn: 0.7525166	test: 0.5282184	best: 0.5285521 (93)	total: 1m 42s	remaining: 52m 22s
95:	learn: 0.7532564	test: 0.5290576	best: 0.5290576 (95)	total: 1m 43s	remaining: 52m 20s
96:	learn: 0.7542427	test: 0.5290576	best: 0.5290576 (95)	total: 1m 44s	remaining: 52m 20s
97:	learn: 0.7547268	test: 0.5309778	best: 0.5309778 (97)	total: 1m 46s	remaining: 52m 21s
98:	learn: 0.7553501	test: 0.5312385	best: 0.5312385 (98)	total: 1m 47s	remaining: 52m 17s
99:	learn: 0.7561885	test: 0.5310582	best: 0.5312385 (98)	total: 1m 48s	remaining: 52m 16s
100:	learn: 0.7566339	test: 0.5316642	best: 0.5316642 (100)	total: 1m 49s	remaining: 52m 16s
101:	learn: 0.7567990	test: 0.5343005	best: 0.5343005 (101)	total: 1m 50s	remaining: 52m 16s
102:	learn: 0.7574471	test: 0.5344299	best: 0.5344299 (102)	total: 1m 51s	remaining: 5

181:	learn: 0.7821961	test: 0.5379662	best: 0.5396208 (154)	total: 3m 13s	remaining: 49m 55s
182:	learn: 0.7820937	test: 0.5382464	best: 0.5396208 (154)	total: 3m 14s	remaining: 49m 54s
183:	learn: 0.7826439	test: 0.5386297	best: 0.5396208 (154)	total: 3m 15s	remaining: 49m 53s
184:	learn: 0.7834203	test: 0.5410452	best: 0.5410452 (184)	total: 3m 16s	remaining: 49m 53s
185:	learn: 0.7838036	test: 0.5398327	best: 0.5410452 (184)	total: 3m 17s	remaining: 49m 52s
186:	learn: 0.7841740	test: 0.5393286	best: 0.5410452 (184)	total: 3m 18s	remaining: 49m 50s
187:	learn: 0.7847646	test: 0.5386854	best: 0.5410452 (184)	total: 3m 20s	remaining: 49m 51s
188:	learn: 0.7852815	test: 0.5403177	best: 0.5410452 (184)	total: 3m 21s	remaining: 49m 53s
189:	learn: 0.7851765	test: 0.5397018	best: 0.5410452 (184)	total: 3m 22s	remaining: 49m 53s
190:	learn: 0.7855176	test: 0.5399442	best: 0.5410452 (184)	total: 3m 23s	remaining: 49m 53s
191:	learn: 0.7863016	test: 0.5397210	best: 0.5410452 (184)	total: 3m 

379:	learn: 0.8181555	test: 0.5472565	best: 0.5473430 (362)	total: 6m 45s	remaining: 46m 33s
380:	learn: 0.8182542	test: 0.5471903	best: 0.5473430 (362)	total: 6m 46s	remaining: 46m 32s
381:	learn: 0.8184713	test: 0.5475644	best: 0.5475644 (381)	total: 6m 47s	remaining: 46m 30s
382:	learn: 0.8190469	test: 0.5475874	best: 0.5475874 (382)	total: 6m 48s	remaining: 46m 30s
383:	learn: 0.8192937	test: 0.5472793	best: 0.5475874 (382)	total: 6m 49s	remaining: 46m 27s
384:	learn: 0.8194384	test: 0.5482037	best: 0.5482037 (384)	total: 6m 50s	remaining: 46m 27s
385:	learn: 0.8192675	test: 0.5484691	best: 0.5484691 (385)	total: 6m 51s	remaining: 46m 25s
386:	learn: 0.8195832	test: 0.5487347	best: 0.5487347 (386)	total: 6m 52s	remaining: 46m 25s
387:	learn: 0.8196982	test: 0.5486683	best: 0.5487347 (386)	total: 6m 53s	remaining: 46m 23s
388:	learn: 0.8196259	test: 0.5486683	best: 0.5487347 (386)	total: 6m 54s	remaining: 46m 22s
389:	learn: 0.8197576	test: 0.5488440	best: 0.5488440 (389)	total: 6m 

468:	learn: 0.8300976	test: 0.5538387	best: 0.5549398 (462)	total: 8m 20s	remaining: 45m 3s
469:	learn: 0.8302118	test: 0.5538387	best: 0.5549398 (462)	total: 8m 22s	remaining: 45m 2s
470:	learn: 0.8301153	test: 0.5535973	best: 0.5549398 (462)	total: 8m 23s	remaining: 45m 1s
471:	learn: 0.8302349	test: 0.5534637	best: 0.5549398 (462)	total: 8m 24s	remaining: 45m
472:	learn: 0.8304443	test: 0.5535563	best: 0.5549398 (462)	total: 8m 25s	remaining: 44m 59s
473:	learn: 0.8304089	test: 0.5533816	best: 0.5549398 (462)	total: 8m 26s	remaining: 44m 58s
474:	learn: 0.8308834	test: 0.5528573	best: 0.5549398 (462)	total: 8m 27s	remaining: 44m 57s
475:	learn: 0.8310818	test: 0.5537310	best: 0.5549398 (462)	total: 8m 28s	remaining: 44m 56s
476:	learn: 0.8310940	test: 0.5534895	best: 0.5549398 (462)	total: 8m 29s	remaining: 44m 56s
477:	learn: 0.8311960	test: 0.5533148	best: 0.5549398 (462)	total: 8m 30s	remaining: 44m 55s
478:	learn: 0.8314421	test: 0.5531812	best: 0.5549398 (462)	total: 8m 31s	rem

...

1619:	learn: 0.9042431	test: 0.5730337	best: 0.5735118 (1618)	total: 27m 57s	remaining: 23m 49s
1620:	learn: 0.9043194	test: 0.5732728	best: 0.5735118 (1618)	total: 27m 58s	remaining: 23m 47s
1621:	learn: 0.9044901	test: 0.5733413	best: 0.5735118 (1618)	total: 27m 59s	remaining: 23m 46s
1622:	learn: 0.9044586	test: 0.5704360	best: 0.5735118 (1618)	total: 27m 59s	remaining: 23m 45s
1623:	learn: 0.9045663	test: 0.5699221	best: 0.5735118 (1618)	total: 28m	remaining: 23m 43s
1624:	learn: 0.9046426	test: 0.5703331	best: 0.5735118 (1618)	total: 28m	remaining: 23m 42s
1625:	learn: 0.9045663	test: 0.5701965	best: 0.5735118 (1618)	total: 28m 1s	remaining: 23m 40s
1626:	learn: 0.9046359	test: 0.5700599	best: 0.5735118 (1618)	total: 28m 2s	remaining: 23m 39s
1627:	learn: 0.9047503	test: 0.5697856	best: 0.5735118 (1618)	total: 28m 2s	remaining: 23m 38s
1628:	learn: 0.9046940	test: 0.5701965	best: 0.5735118 (1618)	total: 28m 3s	remaining: 23m 36s
1629:	learn: 0.9049226	test: 0.5703677	best: 0.57351

1705:	learn: 0.9083704	test: 0.5705051	best: 0.5735118 (1618)	total: 28m 50s	remaining: 21m 52s
1706:	learn: 0.9082818	test: 0.5705397	best: 0.5735118 (1618)	total: 28m 51s	remaining: 21m 51s
1707:	learn: 0.9083956	test: 0.5706073	best: 0.5735118 (1618)	total: 28m 51s	remaining: 21m 49s
1708:	learn: 0.9084651	test: 0.5705734	best: 0.5735118 (1618)	total: 28m 52s	remaining: 21m 48s
1709:	learn: 0.9084651	test: 0.5710866	best: 0.5735118 (1618)	total: 28m 52s	remaining: 21m 47s
1710:	learn: 0.9083640	test: 0.5712918	best: 0.5735118 (1618)	total: 28m 53s	remaining: 21m 45s
1711:	learn: 0.9083513	test: 0.5710866	best: 0.5735118 (1618)	total: 28m 54s	remaining: 21m 44s
1712:	learn: 0.9083513	test: 0.5705051	best: 0.5735118 (1618)	total: 28m 54s	remaining: 21m 43s
1713:	learn: 0.9083070	test: 0.5698204	best: 0.5735118 (1618)	total: 28m 55s	remaining: 21m 41s
1714:	learn: 0.9084967	test: 0.5697187	best: 0.5735118 (1618)	total: 28m 55s	remaining: 21m 40s
1715:	learn: 0.9084904	test: 0.5691349	b

1791:	learn: 0.9117545	test: 0.5722123	best: 0.5736083 (1771)	total: 29m 43s	remaining: 20m 2s
1792:	learn: 0.9116789	test: 0.5727912	best: 0.5736083 (1771)	total: 29m 44s	remaining: 20m 1s
1793:	learn: 0.9117923	test: 0.5725528	best: 0.5736083 (1771)	total: 29m 45s	remaining: 20m
1794:	learn: 0.9117290	test: 0.5727912	best: 0.5736083 (1771)	total: 29m 45s	remaining: 19m 58s
1795:	learn: 0.9117729	test: 0.5723143	best: 0.5736083 (1771)	total: 29m 46s	remaining: 19m 57s
1796:	learn: 0.9118362	test: 0.5724845	best: 0.5736083 (1771)	total: 29m 46s	remaining: 19m 56s
1797:	learn: 0.9117606	test: 0.5724163	best: 0.5736083 (1771)	total: 29m 47s	remaining: 19m 55s
1798:	learn: 0.9117667	test: 0.5726547	best: 0.5736083 (1771)	total: 29m 48s	remaining: 19m 53s
1799:	learn: 0.9118045	test: 0.5726547	best: 0.5736083 (1771)	total: 29m 48s	remaining: 19m 52s
1800:	learn: 0.9118923	test: 0.5726547	best: 0.5736083 (1771)	total: 29m 49s	remaining: 19m 51s
1801:	learn: 0.9120311	test: 0.5724845	best: 0

1877:	learn: 0.9147550	test: 0.5714286	best: 0.5736083 (1771)	total: 30m 37s	remaining: 18m 17s
1878:	learn: 0.9147867	test: 0.5711555	best: 0.5736083 (1771)	total: 30m 37s	remaining: 18m 16s
1879:	learn: 0.9148995	test: 0.5715309	best: 0.5736083 (1771)	total: 30m 38s	remaining: 18m 15s
1880:	learn: 0.9150064	test: 0.5710529	best: 0.5736083 (1771)	total: 30m 38s	remaining: 18m 13s
1881:	learn: 0.9151051	test: 0.5713603	best: 0.5736083 (1771)	total: 30m 39s	remaining: 18m 12s
1882:	learn: 0.9150300	test: 0.5712579	best: 0.5736083 (1771)	total: 30m 40s	remaining: 18m 11s
1883:	learn: 0.9150675	test: 0.5707801	best: 0.5736083 (1771)	total: 30m 40s	remaining: 18m 10s
1884:	learn: 0.9150675	test: 0.5707119	best: 0.5736083 (1771)	total: 30m 41s	remaining: 18m 9s
1885:	learn: 0.9151051	test: 0.5712920	best: 0.5736083 (1771)	total: 30m 42s	remaining: 18m 8s
1886:	learn: 0.9149983	test: 0.5709165	best: 0.5736083 (1771)	total: 30m 42s	remaining: 18m 6s
1887:	learn: 0.9149231	test: 0.5713944	best

1963:	learn: 0.9179148	test: 0.5720753	best: 0.5736083 (1771)	total: 31m 31s	remaining: 16m 37s
1964:	learn: 0.9179954	test: 0.5722116	best: 0.5736083 (1771)	total: 31m 32s	remaining: 16m 36s
1965:	learn: 0.9180271	test: 0.5720076	best: 0.5736083 (1771)	total: 31m 32s	remaining: 16m 35s
1966:	learn: 0.9178570	test: 0.5721779	best: 0.5736083 (1771)	total: 31m 33s	remaining: 16m 34s
1967:	learn: 0.9178888	test: 0.5725864	best: 0.5736083 (1771)	total: 31m 34s	remaining: 16m 33s
1968:	learn: 0.9178139	test: 0.5725528	best: 0.5736083 (1771)	total: 31m 34s	remaining: 16m 32s
1969:	learn: 0.9180214	test: 0.5727240	best: 0.5736083 (1771)	total: 31m 35s	remaining: 16m 30s
1970:	learn: 0.9179466	test: 0.5726557	best: 0.5736083 (1771)	total: 31m 35s	remaining: 16m 29s
1971:	learn: 0.9179148	test: 0.5728943	best: 0.5736083 (1771)	total: 31m 36s	remaining: 16m 28s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.5736082966
bestIteration = 1771

Shrink model to first 1772 iterati

In [8]:
train_pool=Pool(X_test1, y_test1,cat_features=categorical_features_indices)
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_test1.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

stocn_value_counts_all: 7.83736260934982
bacno_cano_nunique: 4.967941377174668
kmeans_all_feature3: 3.534430284921202
cano_locdt_skew: 3.2645915336626197
txkey: 2.760187696452213
kmeans_all_feature5: 2.6768021903343873
stocn_cano_nunique: 2.475305634760855
ecfg: 2.4551860502393232
cano_lastlocdt2: 2.4071256721345655
cano_conam_skew: 2.3906531385388265
bacno_locdt_kurt: 2.008918123389369
bacno_locdt_skew: 1.9339004285571884
isolationtree_all_feature: 1.7969433339517136
etymd: 1.7662675705851665
cano_conam_kurt: 1.7629369224941882
stocn: 1.7507026827771601
mchno: 1.676806702566594
stocn_bacno_nunique: 1.5765487636951006
cano_locdt_kurt: 1.539750901484057
contp_ratio: 1.4858260088790327
hcefg_ratio: 1.4682975377373204
stocn_value_counts: 1.4583734428423494
bacno_min_conam: 1.4524976860543704
cano_scale_conam: 1.350541332994721
pca2_feature_1: 1.3258185132893472
etymd_value_counts: 1.2987176318591622
conam: 1.2827745717071455
conam_log: 1.282101567922175
mcc_value_counts: 1.24462839213701


In [9]:
## 理論上th設0.5一定是最好的？

y_test1_pred = model.predict_proba(X_test1,verbose=True)[:,1]
th=0.35

y_test1_pred[y_test1_pred>th]=1
y_test1_pred[y_test1_pred<=th]=0
print('F1 score',f1_score(y_test1, y_test1_pred))

tn, fp, fn, tp = confusion_matrix(y_test1, y_test1_pred).ravel()
print('tn fp fn tp')
print(tn, fp, fn, tp)
print('Percision', tp/(tp+fp))
print('Recall',tp/(tp+fn))


F1 score 0.587221204611572
tn fp fn tp
500440 1407 2424 2725
Percision 0.659486931268151
Recall 0.5292289765002913


In [10]:
model = CatBoostClassifier(**param_cat)

model.fit(
    X_train_all, y_train_all,
    cat_features=categorical_features_indices,    
    silent=False
)


0:	learn: 0.4291968	total: 835ms	remaining: 41m 42s
1:	learn: 0.4791178	total: 1.66s	remaining: 41m 22s
2:	learn: 0.4853770	total: 2.55s	remaining: 42m 25s
3:	learn: 0.4913070	total: 3.39s	remaining: 42m 22s
4:	learn: 0.5043872	total: 4.19s	remaining: 41m 49s
5:	learn: 0.5067109	total: 5.11s	remaining: 42m 29s
6:	learn: 0.5138966	total: 6.1s	remaining: 43m 29s
7:	learn: 0.5231149	total: 7.05s	remaining: 43m 56s
8:	learn: 0.5309826	total: 7.94s	remaining: 43m 58s
9:	learn: 0.5283441	total: 8.79s	remaining: 43m 49s
10:	learn: 0.5341408	total: 9.63s	remaining: 43m 36s
11:	learn: 0.5345750	total: 10.5s	remaining: 43m 23s
12:	learn: 0.5458477	total: 11.2s	remaining: 42m 53s
13:	learn: 0.5567975	total: 12s	remaining: 42m 33s
14:	learn: 0.5583135	total: 12.8s	remaining: 42m 20s
15:	learn: 0.5663027	total: 13.7s	remaining: 42m 27s
16:	learn: 0.5760358	total: 14.5s	remaining: 42m 29s
17:	learn: 0.5785982	total: 15.4s	remaining: 42m 26s
18:	learn: 0.5809691	total: 16.1s	remaining: 42m 10s
19:	le

154:	learn: 0.7330236	total: 2m 8s	remaining: 39m 23s
155:	learn: 0.7332989	total: 2m 9s	remaining: 39m 21s
156:	learn: 0.7343477	total: 2m 10s	remaining: 39m 20s
157:	learn: 0.7345371	total: 2m 11s	remaining: 39m 20s
158:	learn: 0.7349874	total: 2m 12s	remaining: 39m 19s
159:	learn: 0.7351711	total: 2m 12s	remaining: 39m 17s
160:	learn: 0.7358382	total: 2m 13s	remaining: 39m 17s
161:	learn: 0.7360041	total: 2m 14s	remaining: 39m 16s
162:	learn: 0.7364323	total: 2m 15s	remaining: 39m 15s
163:	learn: 0.7367307	total: 2m 16s	remaining: 39m 15s
164:	learn: 0.7378080	total: 2m 16s	remaining: 39m 13s
165:	learn: 0.7373417	total: 2m 17s	remaining: 39m 12s
166:	learn: 0.7376454	total: 2m 18s	remaining: 39m 11s
167:	learn: 0.7381299	total: 2m 19s	remaining: 39m 11s
168:	learn: 0.7385838	total: 2m 20s	remaining: 39m 10s
169:	learn: 0.7389329	total: 2m 21s	remaining: 39m 10s
170:	learn: 0.7393181	total: 2m 22s	remaining: 39m 9s
171:	learn: 0.7389937	total: 2m 22s	remaining: 39m 8s
172:	learn: 0.

305:	learn: 0.7685310	total: 4m 12s	remaining: 37m 4s
306:	learn: 0.7688163	total: 4m 13s	remaining: 37m 3s
307:	learn: 0.7692696	total: 4m 14s	remaining: 37m 3s
308:	learn: 0.7693301	total: 4m 15s	remaining: 37m 2s
309:	learn: 0.7693646	total: 4m 16s	remaining: 37m 1s
310:	learn: 0.7699299	total: 4m 16s	remaining: 37m
311:	learn: 0.7698738	total: 4m 17s	remaining: 37m
312:	learn: 0.7698825	total: 4m 18s	remaining: 36m 59s
313:	learn: 0.7702403	total: 4m 19s	remaining: 36m 58s
314:	learn: 0.7704173	total: 4m 20s	remaining: 36m 58s
315:	learn: 0.7706499	total: 4m 21s	remaining: 36m 57s
316:	learn: 0.7705203	total: 4m 22s	remaining: 36m 57s
317:	learn: 0.7705543	total: 4m 22s	remaining: 36m 57s
318:	learn: 0.7707353	total: 4m 23s	remaining: 36m 55s
319:	learn: 0.7713374	total: 4m 24s	remaining: 36m 54s
320:	learn: 0.7713934	total: 4m 25s	remaining: 36m 52s
321:	learn: 0.7714582	total: 4m 25s	remaining: 36m 51s
322:	learn: 0.7716734	total: 4m 26s	remaining: 36m 49s
323:	learn: 0.7719357	t

456:	learn: 0.7891991	total: 6m 15s	remaining: 34m 51s
457:	learn: 0.7893875	total: 6m 16s	remaining: 34m 50s
458:	learn: 0.7893860	total: 6m 17s	remaining: 34m 49s
459:	learn: 0.7895643	total: 6m 18s	remaining: 34m 48s
460:	learn: 0.7898549	total: 6m 19s	remaining: 34m 47s
461:	learn: 0.7896753	total: 6m 19s	remaining: 34m 46s
462:	learn: 0.7898432	total: 6m 20s	remaining: 34m 45s
463:	learn: 0.7897439	total: 6m 21s	remaining: 34m 44s
464:	learn: 0.7897877	total: 6m 22s	remaining: 34m 43s
465:	learn: 0.7896738	total: 6m 22s	remaining: 34m 42s
466:	learn: 0.7898973	total: 6m 23s	remaining: 34m 41s
467:	learn: 0.7899734	total: 6m 24s	remaining: 34m 40s
468:	learn: 0.7899398	total: 6m 25s	remaining: 34m 39s
469:	learn: 0.7900858	total: 6m 26s	remaining: 34m 39s
470:	learn: 0.7903311	total: 6m 27s	remaining: 34m 38s
471:	learn: 0.7902653	total: 6m 27s	remaining: 34m 37s
472:	learn: 0.7901529	total: 6m 28s	remaining: 34m 37s
473:	learn: 0.7902886	total: 6m 29s	remaining: 34m 36s
474:	learn

KeyboardInterrupt: 

In [ ]:
y_test_pred_cat = model.predict_proba(X_test_all)[:,1]

print(X_test_all.index)

th=0.37
y_test_pred_cat[y_test_pred_cat>th]=1
y_test_pred_cat[y_test_pred_cat<=th]=0

## write csv

In [ ]:
import time
print(time.localtime( time.time() ))
a = time.localtime( time.time() )
print(str(a.tm_mon)+str(a.tm_mday))

In [ ]:
result = y_test_pred_cat
test_data_txkey = all_data[all_data['locdt']>90]['txkey'].values

print('{}: prediction positive ratio'.format(result.sum()/result.shape[0]))
print('{}: training positive ratio'.format(y_train_all.sum()/y_train_all.shape[0]))

import time
t_now = time.localtime( time.time() )
t = str(t_now.tm_mon)+str(t_now.tm_mday)+str(t_now.tm_hour)+str(t_now.tm_min)
print('Now:',t)

submit_file_name='submit_cat_th{}_time{}.csv'.format(th,t)
import csv
with open('../prediction/{}'.format(submit_file_name),'w') as f:
    writer = csv.writer(f)
    writer.writerow(['txkey','fraud_ind'])
    for i in range(result.shape[0]):
        writer.writerow([test_data_txkey[i], result[i]])
        
with open('../prediction/log.txt','w') as f:
    print('{}'.format(submit_file_name),file=f)
    print('delete_list:\n{}'.format(delete_list),file=f)

## 贝叶斯调参
* http://nohup.cc/article/258/
* https://github.com/fmfn/BayesianOptimization

Results:
{'params': {'bagging_temperature': 1.0,
  'learning_rate': 0.2,
  'reg_lambda': 17.267046492047776},
 'target': 0.5339179149920837}

In [ ]:
# from bayes_opt import BayesianOptimization
# def cat_train(bagging_temperature, reg_lambda, learning_rate):
#     params = {
#         'iterations':2000,
#         'depth':6,
#         'bagging_temperature':bagging_temperature,
#         'reg_lambda':reg_lambda,
#         'learning_rate':learning_rate,
#         'loss_function':'Logloss',
#         'eval_metric':'F1',
#         'random_seed':random_seed,
#         'verbose':30
#     }
 
#     model = CatBoostClassifier(**params)
#     # 评价数据集是验证集，评价指标是AUC
#     model.fit(X_train1, y_train1,\
#               eval_set=(X_test1, y_test1),\
#               cat_features=categorical_features_indices,\
#               early_stopping_rounds=200) 
     
#     print(params)
#     score_max = model.best_score_.get('validation').get('F1')
#     return score_max
 
# cat_opt = BayesianOptimization(cat_train, 
#                            {
#                               'bagging_temperature': (1, 50),  
#                               'reg_lambda': (1, 200),
#                               'learning_rate':(0.05, 0.2)
#                             })
 
# cat_opt.maximize(n_iter=15, init_points=random_seed)
# cat_opt.max

## Permutation importance

In [ ]:
# def permutation_scoring(model,X,y):
#     y_pred = model.predict(X)
#     return f1_score(y, y_pred)

# import eli5
# from eli5.sklearn import PermutationImportance

# print(y_test1.sum())
# print(model.score(X_test1, y_test1))
# perm = PermutationImportance(model, random_state=random_seed,scoring=permutation_scoring,n_iter=50).fit(X_test1, y_test1)
# feature_importance1 = pd.DataFrame({'feature':X_test1.columns.tolist(),'importance':perm.feature_importances_})
# delete_col1 = feature_importance1.iloc[:,0][(feature_importance1['importance'].values)<=0.0000]
# print(delete_col1)

# feature_importance1.sort_values(by=['importance'],inplace=True)
# top_col = feature_importance1
# for i in range(top_col.shape[0]):
#     print(top_col.iloc[i].values)

In [ ]:
# from eli5.catboost import explain_weights_catboost
# explain_weights_catboost(model, vec=None, top=70, importance_type='PredictionValuesChange', feature_names=None, pool=None)

## 將使用者分類來訓練模型:
1. 根據cano的個數分類
2. 根據txkey的個數分類
3. 根據stocn的眾數分類

murmur: 感覺這方法怪怪的,tree模型應該就能涵蓋進去

In [ ]:
## txkey
txkey_qcut_id = pd.qcut(all_data['bacno_txkey_nunique'],3,labels=[0,1,2])

# print(all_data['bacno_txkey_nunique'])
# print(txkey_qcut_id)
models=[]
for i in range(3):
    print(i)

## Anomaly detection
* one class svm
* isolation tree
* replicator NN
* Kmeans?
* KNN(take too much time)

## 異常偵測
wiki<br>
https://zh.wikipedia.org/wiki/%E5%BC%82%E5%B8%B8%E6%A3%80%E6%B5%8B#cite_note-9

因為盜刷很可能都是outlier，一般的機器學習方法在outlier上表現會很差，因此可以用來解釋為什麼會train不好的原因
http://www.cainiaoxueyuan.com/suanfa/7017.html<br>
https://towardsdatascience.com/outlier-detection-with-isolation-forest-3d190448d45e<br>
https://medium.com/@cyeninesky3/oneclass-svm-%E7%95%B0%E5%B8%B8%E6%AA%A2%E6%B8%AC%E4%BB%BB%E5%8B%99-anomaly-detection-%E7%9A%84%E7%AE%97%E6%B3%95%E7%90%86%E8%A7%A3%E8%88%87%E5%AF%A6%E8%B8%90-cf5f0bbb01c0<br>

isolation tree<br>
https://zhuanlan.zhihu.com/p/25040651
https://scikit-learn.org/stable/auto_examples/ensemble/plot_isolation_forest.html
https://towardsdatascience.com/outlier-detection-with-extended-isolation-forest-1e248a3fe97b

oneclass svm<br>
https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html

Replicator NN<br>
https://togaware.com/papers/dawak02.pdf

one class kmeans?<br>
https://ai100-2.cupoy.com/mission/D57

## 製作特徵
XGB, LGB, PCA, Isolation Forest, Kmean距離？, oneclass SVM?
當作新feature

In [ ]:
# import xgboost as xgb
# param_dist_xgb = {'learning_rate':0.01, #默认0.3
#               'n_estimators':1000, #树的个数
# #               'max_depth':5,
# #               'min_child_weight':1,
# #               'gamma':0.2,
# #               'subsample':0.8,
# #               'colsample_bytree':0.8,
# #               'objective': 'binary:logistic', #逻辑回归损失函数
# #               'nthread':4,  #cpu线程数
# #               'scale_pos_weight':1,
#               'seed':random_seed}  #随机种子

# evals_result = {}

# xgb_clf = xgb.XGBClassifier(**param_dist_xgb)
# xgb_clf.fit(X_train, y_train,
#         eval_set=[(X_train, y_train),(X_test, y_test)],
#         eval_metric=lgb_f1_score,
#         early_stopping_rounds=600,
#         verbose=True,
# #         callbacks=[xgb.record_evaluation(evals_result)]
#         )

# print('F1',f1_score(y_test, xgb_clf.predict(X_test)))
# xgb_X_train = xgb_clf.apply(X_train)
# xgb_X_test = xgb_clf.apply(X_test)

## Train on LGB(未調參數)(效果不好)

In [ ]:
print(delete_list)
print('Training num',X_train1.shape)
print('positive label ratio-train',y_train1.sum()/y_train1.shape[0])
print('positive label ratio-test',y_test1.sum()/y_test1.shape[0])

def lgb_f1_score(y_true, y_pred):
    y_pred = np.round(y_pred) # scikits f1 doesn't like probabilities
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    print()
    print('tn, fp, fn, tp')
    print(tn, fp, fn, tp)
    return 'f1', f1_score(y_true, y_pred), True

param_dist_lgb = {
#                   'num_leaves':45, 
#                   'max_depth':5, 
                  'learning_rate':0.1, 
                  'n_estimators':600,
                  'objective': 'binary',
#                   'subsample': 1, 
#                   'colsample_bytree': 0.5, 
#                   'lambda_l1': 0.1,
#                   'lambda_l2': 0,
#                   'min_child_weight': 1,
                  'random_state': random_seed,
                 }
evals_result = {}

lgb_clf = LGBMClassifier(**param_dist_lgb)
lgb_clf.fit(X_train1, y_train1,
        eval_set=[(X_train1, y_train1),(X_test1, y_test1)],
        eval_metric=lgb_f1_score,
        early_stopping_rounds=50,
        verbose=True,
        callbacks=[lgb.record_evaluation(evals_result)]
        )
y_test_pred = lgb_clf.predict(X_test1)
print('F1',f1_score(y_test1, y_test_pred))
tn, fp, fn, tp = confusion_matrix(y_test1, y_test_pred).ravel()
print(tn, fp, fn, tp)

In [ ]:
print('Plotting metrics recorded during training...')
ax = lgb.plot_metric(evals_result, metric='f1')
plt.show()

print('Plotting feature importances...')
ax = lgb.plot_importance(lgb_clf, max_num_features=30)
plt.show()

print('Plotting 4th tree...')  # one tree use categorical feature to split
ax = lgb.plot_tree(lgb_clf, tree_index=3, figsize=(15, 15), show_info=['split_gain'])
plt.show()

print('Plotting 4th tree with graphviz...')
graph = lgb.create_tree_digraph(lgb_clf, tree_index=3, name='Tree4')
graph.render(view=True)

In [ ]:
feature_importance = np.stack([X_train1.columns.values,lgb_clf.feature_importances_]).transpose()
feature_importance = pd.DataFrame(feature_importance,columns=['feature_name','importance'])
feature_importance.sort_values(by=['importance'],inplace=True,ascending=False)
print(feature_importance)

## PCA visualization in one person who has fraud data

In [ ]:
from sklearn.decomposition import PCA
def PCA_plot(x,label):
    x = x.drop(columns=delete_list)
    
    ## 應該先轉dummy,標準化,再PCA
#     dummy_list=['contp','etymd','stscd','hcefg']
#     dummy_list2=['stocn','scity','csmcu']#'mchno','acqic','mcc',
#     x[dummy_list] = x[dummy_list].astype(object)
#     x[dummy_list2] = x[dummy_list2].astype(object)
#     x = pd.get_dummies(x)    
    
    from sklearn.preprocessing import StandardScaler 
    stdsc = StandardScaler() 
    x = stdsc.fit_transform(x)
    print(x.shape,label.sum())

    PCA_model = PCA(n_components=2)
    train_data_pca = PCA_model.fit_transform(x)
    train_data_pca1 = train_data_pca[label==1]
    train_data_pca0 = train_data_pca[label==0]
    
    plt.clf()
    plt.figure(figsize=(10,10))
    plt.scatter(train_data_pca1[:, 0], train_data_pca1[:, 1], c='r',label='fraud transaction',s=100)
    plt.scatter(train_data_pca0[:, 0], train_data_pca0[:, 1], c='b',label='normal transaction',s=3)
    plt.legend()
    plt.show()
    
bacno_hasfraud = all_data[all_data['fraud_ind']==1]['bacno'].unique()
print(bacno_hasfraud.shape[0])
print(all_data[all_data['fraud_ind']==1].shape[0])

for i in range(bacno_hasfraud.shape[0]):
    if all_data[all_data['bacno']==bacno_hasfraud[i]].shape[0]>300:
        print('Ploting PCA on bacno-{}'.format(bacno_hasfraud[i]))
        PCA_plot(all_data[all_data['bacno']==bacno_hasfraud[i]],all_data[all_data['bacno']==bacno_hasfraud[i]]['fraud_ind'])

## TSNE, Kmeans作圖?

## Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

c_ratio = y_train.sum()/y_train.shape[0]
# fit the model
clf = IsolationForest(behaviour='new', max_samples=0.8, max_features=1,
                      random_state=random_seed, contamination=c_ratio)
clf.fit(X_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

y_pred_test2 = -y_pred_test
y_pred_test2[y_pred_test2==-1]=0
y_pred_test2.sum()

y_pred_train2 = -y_pred_train
y_pred_train2[y_pred_train2==-1]=0
y_pred_train2.sum()

from sklearn.metrics import f1_score
print(f1_score(y_train, y_pred_train2))
print(f1_score(y_test, y_pred_test2))

isolationtree_X_train = clf.score_samples(X_train)
isolationtree_X_test = clf.score_samples(X_test)

print(isolationtree_X_train)

## One class SVM

In [ ]:
from sklearn import svm

clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma='scale',verbose=True, random_state=random_seed)
clf.fit(X_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_test.sum()

y_pred_train2 = -y_pred_train
y_pred_train2[y_pred_train2==-1]=0
y_pred_train2.sum()

from sklearn.metrics import f1_score
print(f1_score(y_train, y_pred_train2))
print(f1_score(y_test, y_pred_test2))

svm_X_train = clf.score_samples(X_train)
svm_X_test = clf.score_samples(X_test)

print(isolationtree_X_train)

In [ ]:
# cano_lastlocdt
# bacno_lastlocdt
# bacno_cano_nunique



In [ ]:
# bacno_etymd_nunique
# bacno_contp_nunique

In [ ]:
# 用hinge loss(當SVM)

In [ ]:
# X_train['cents']
# encoding data

# GroupKfold
# vanilla KFold